In [ ]:
%load_ext autoreload
%autoreload 2
from visualize import label_shot_chains
from data import load_and_process_soccer_events, load_and_process_metadata
from label import pos_labeling

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
event_df, players_df = load_and_process_soccer_events("/home/soccerdata/FIFA_WorldCup_2022/Event Data")
metadata_df = load_and_process_metadata("/home/soccerdata/FIFA_WorldCup_2022/Metadata")

In [3]:
pos_chains = pos_labeling(event_df, chain_len=6)

In [95]:
accepted_chains = label_shot_chains(
    chains=pos_chains,                
    chains_range=(0, 3),  
    event_df=event_df,
    players_df=players_df,
    metadata_df=metadata_df,
    output_dir="/home/soccerdata/frames",
    show_video=True,
    interval=1000
)

In [94]:
accepted_chains

[[109, 110, 111, 112, 113, 114, 115], [245], [545]]